In [ ]:
# upload file
from google.colab import files
uploaded = files.upload()

Saving TrainingData.csv to TrainingData.csv


In [ ]:
# import packages
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
# load data and transform to array
df = pd.read_csv("TrainingData.csv", header = 0)
df.head(10)

,rowID,chunkID,position_within_chunk,month_most_common,weekday,hour,Solar.radiation_64,WindDirection..Resultant_1,WindDirection..Resultant_1018,WindSpeed..Resultant_1,...,target_4_6006,target_4_8003,target_5_6006,target_7_57,target_8_57,target_8_4002,target_8_6004,target_8_8003,target_9_4002,target_9_8003
0,1,1,1,10,Saturday,21,0.01,117.0,187.0,0.3,...,1.748424,NaN,NaN,5.130631,1.341606,2.138792,3.013752,NaN,5.679280,NaN
1,2,1,2,10,Saturday,22,0.01,231.0,202.0,0.5,...,2.144120,NaN,NaN,5.130631,1.195779,2.722099,3.888712,NaN,7.426751,NaN
2,3,1,3,10,Saturday,23,0.01,247.0,227.0,0.5,...,1.932469,NaN,NaN,5.136395,1.409658,3.110970,3.888712,NaN,7.683732,NaN
3,4,1,4,10,Sunday,0,0.01,219.0,218.0,0.2,...,2.088907,NaN,NaN,5.217102,1.477711,2.041574,3.208188,NaN,4.831243,NaN
4,5,1,5,10,Sunday,1,0.01,2.0,216.0,0.2,...,2.604232,NaN,NaN,5.217102,1.458267,2.138792,3.499841,NaN,4.625658,NaN
5,6,1,6,10,Sunday,2,0.01,288.0,2.0,0.3,...,2.687052,NaN,NaN,5.170984,1.604094,2.236010,3.305406,NaN,5.833469,NaN
6,7,1,7,10,Sunday,3,0.01,330.0,8.0,0.3,...,2.677850,NaN,NaN,2.951554,1.555485,1.652703,2.430445,NaN,3.854715,NaN
7,8,1,8,10,Sunday,4,0.01,316.0,4.0,0.8,...,2.549019,NaN,NaN,2.951554,1.711033,1.458267,1.361049,NaN,2.595508,NaN
8,9,1,9,10,Sunday,5,0.01,285.0,342.0,0.7,...,2.226941,NaN,NaN,2.928495,1.623537,1.361049,1.263832,NaN,2.621206,NaN
9,10,1,10,10,Sunday,6,0.05,337.0,352.0,1.3,...,1.996885,NaN,NaN,1.815897,1.380493,1.263832,1.166614,NaN,2.004452,NaN


In [ ]:
# Naive prediction for model evaluation
# divide data into chunks
def to_chunks(df):
  chunk_id = np.unique(df['chunkID'])
  chunks = dict()
  for id in chunk_id:
    chunks[id] = df[df['chunkID']==id]
  return chunks
# split each chunk into train data and test data (first five days for train and the rest three days test)
def split_train_test(chunks):
  train = dict()
  test = dict()
  # first five days of hourly observations for train
  cut_point = 5*24
  for id, data in chunks.items():
    train_rows = data[data['position_within_chunk'] <= cut_point]
    test_rows = data[data['position_within_chunk'] > cut_point]
    if len(train_rows)==0 or len(test_rows)==0:
      print(f"drop chunk {id}")
      continue
    index = [1,2,5] + [x for x in range(56, 95)]
    train[id] = train_rows.iloc[:,index]
    test[id] = test_rows.iloc[:,index]
  return train, test
# return a list of relative forecast lead times
def get_lead_times():
  return [1,2,3,4,5,10,17,24,48,72]
# convert the rows in a test chunk to forecast
def to_forecasts(test):
  forecasts = list()
  lead_times = get_lead_times()
  cut_point = 24*5
  for tau in lead_times:
    offset = cut_point+tau
    for id, data in test.items():
      row_for_tau = data[data['position_within_chunk']==offset]
      if len(row_for_tau)==0:
      # create a mock row [chunkID, position, hour]+[nan...]
        row = [id, offset, np.nan] + [np.nan for _ in range(39)]
        forecasts.append(row)
      else:
        forecasts.append(row_for_tau.values.tolist()[0])
  return forecasts

In [ ]:
chunks = to_chunks(df)
train,test = split_train_test(chunks)

In [ ]:
forecasts = to_forecasts(test)
test_array = np.array(forecasts)

In [ ]:
train_array=[]
for id, values in train.items():
  list_value = values.values.tolist()
  train_array.extend(list_value)
train_array = np.array(train_array)

In [ ]:
# save training data and test data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
np.savetxt('/content/drive/MyDrive/AirPollutionPrediction/Naive Prediction/train.txt', train_array)
np.savetxt('/content/drive/MyDrive/AirPollutionPrediction/Naive Prediction/test.txt', test_array)

In [1]:
# save the value into chunks
def to_chunks(data):
  chunks = list()
  chunk_id = np.unique(data[:,0])
  for id in chunk_id:
    chunks.append(data[data[:,0]== id,:])
  return chunks

# chunk_train is data for single chunk, col_ix is the variable index
def value_to_series(chunk_train,col_ix, n_length = 24*5):
  series = [np.nan for _ in range(n_length)]
  for i in range(len(chunk_train)):
    position = int(chunk_train[i,1]-1)
    series[position] =  chunk_train[i,col_ix]
  return series

# interpolate hours for each chunk
def interpolate_hours(hours):
  ix = -1
  for i in range(len(hours)):
    if not np.isnan(hours[i]):
      ix = i
      break
  # forward
  hour =  hours[ix]
  for i in range(ix+1,len(hours),1):
    hour+=1
    if np.isnan(hours[i]):
      hours[i] = hour % 24
  # backward
  hour = hours[ix]
  for i in range(ix-1,-1,-1):
    hour-=1
    if np.isnan(hours[i]):
      hours[i] = hour % 24

# impute missing values cross chunks
def impute_missing_values(train_chunks,series, hours,col_ix):
    if np.count_nonzero(np.isnan(series))> 0:
      imputed_series = []
      for i in range(len(series)):
        if np.isnan(series[i]):
          all_rows = []
          for chunk in train_chunks:
           [all_rows.append(row) for row in chunk[chunk[:,2]== hours[i],col_ix]]
          all_rows = np.array(all_rows)
          value = np.nanmedian(all_rows)
          if np.isnan(value):
            value = 0.0
          imputed_series.append(value)
        else:
          imputed_series.append(series[i])
      return imputed_series
    else:
      return series

# get lead times
def get_lead_times():
  return [1,2,3,4,5,10,17,24,48,72]

# prepare data to supervised learning given n_lags and lead_time
def stack_series_to_supervised(series,n_lags,lead_time):
  X=list()
  y=list()
  for i in range(len(series)):
    end_ix = i+n_lags+lead_time-1
    if end_ix >= len(series):
      break
    else:
      seq_x = series[i:i+n_lags, :]
      seq_y = series[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

# prepare target column to supervised learning for given lead_time and chunk
def target_to_supervised(chunks,chunk_train,n_lags,lead_time,n_var=39):
  data = list()
  for i in range(n_var):
    col_ix = 3+i
    # get the target series and hours
    series = value_to_series(chunk_train, col_ix)
    hours = value_to_series(chunk_train, 2)
    # interpolate hours
    interpolate_hours(hours)
    # impute missing values
    imputed_series = impute_missing_values(chunks,series, hours, col_ix)
    imputed_series = np.array(imputed_series).reshape((len(imputed_series), 1))
    data.append(imputed_series)
  data = np.hstack(data)
  # prepared test sample
  test_sample = data[-n_lags:,:]
  # for each lead_time, series_to_superivsed
  X, y = stack_series_to_supervised(data,n_lags,lead_time)
  return X,y,test_sample

# prepare training data for one given lead_time: put all chunks data together
def prepare_training(chunks, n_lags,lead_time):
  X_train_prepared = list()
  y_train_prepared = list()
  test_prepared = [[] for i in range(len(chunks))]
  for i in range(len(chunks)):
    print(f'chunk{i} starts preparing')
    chunk_train = chunks[i]
    X,y,test_sample = target_to_supervised(chunks,chunk_train,n_lags,lead_time)
    X_train_prepared.extend(X)
    y_train_prepared.extend(y)
    test_prepared[i] = test_sample
  return np.array(X_train_prepared),np.array(y_train_prepared),np.array(test_prepared)

In [ ]:
import tensorflow as tf

In [ ]:
def fit_Vaniliia_LSTM(n_lags, n_features):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(50, activation='relu', return_sequences = True, input_shape=(n_lags,n_features)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_features))
  model.compile(optimizer='adam', loss='mse')
  return model

In [ ]:
def fit_stack_LSTM(n_lags, n_features):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(50, activation='relu', return_sequences = True, input_shape=(n_lags,n_features)))
  model.add(tf.keras.layers.LSTM(50, activation='relu', return_sequences = True, input_shape=(n_lags,n_features)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_features))
  model.compile(optimizer='adam', loss='mae')
  return model

In [ ]:
def fit_CNN(n_lags, n_features):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv1D(filters = 64,kernel_size = 2, activation='relu',input_shape=(n_lags,n_features)))
  model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(50,activation='relu'))
  model.add(tf.keras.layers.Dense(n_features))
  model.compile(optimizer='adam', loss='mae')
  return model

In [ ]:
def fit_CNN_LSTM(n_lags, n_features):
  model= tf.keras.models.Sequential()
  model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None,n_lags, n_features)))
  model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(pool_size=2)))
  model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
  model.add(tf.keras.layers.LSTM(50,activation='relu'))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_features))
  model.compile(optimizer = 'adam', loss='mae')
  return model

In [ ]:
# return a list of prediction for each chunk
def make_predictions(fitted_model,test):
  prediction_chunks = list()
  for i in range(len(test)):
    prediction = fitted_model.predcit(test[i])
    prediction_chunks.append(prediction)
  return prediction_chunks

In [ ]:
# calculate the error between an actual and prediced value
def calculate_error(actual, predicted):
  if np.isnan(predicted):
    return abs(actual)
  else:
    return abs(actual - predicted)

# evaluate a forecast for each leadtime
def evaluate_forecasts(prediction_chunks, test_chunks,lead_time,n_var=39):
  error_sum = list()
  position_in_chunk = 120 + lead_time
  for i in range(len(test_chunks)):
    position = test_chunks[i][:,1]== position_in_chunk
    actual = test_chunks[i][position,3:3+n_var]
    prediction = prediction_chunks[i]
    if np.isnan(actual):
      continue
    error = calculate_error(actual, prediction)
    error_sum.append(error)
    n_chunks += 1
  return error_sum, n_chunks

In [ ]:
#  read file from google colab drive
from google.colab import drive
drive.mount('/content/drive')
train_array = np.loadtxt('/content/drive/MyDrive/AirPollutionPrediction/Naive Prediction/train.txt')
test_array = np.loadtxt('/content/drive/MyDrive/AirPollutionPrediction/Naive Prediction/test.txt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_chunks = to_chunks(train_array)
x_train, y_train, x_test = prepare_training(train_chunks, 4, 5)

In [ ]:
test_chunks = to_chunks(test_array)